In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector

In [12]:
def load_data():
    orders_df = pd.read_csv('../ShopEaseData/orders.csv')
    products_df = pd.read_csv('../ShopEaseData/products.csv')
    
    # Merge orders with products to get price information
    merged_df = pd.merge(orders_df, products_df, on='product_id', how='left')
    return merged_df

In [13]:
def clean_data(df):
    # Drop duplicates
    df = df.drop_duplicates()
    
    # Handle missing values
    df = df.dropna()
    
    # Convert data types
    df['order_date'] = pd.to_datetime(df['order_date'])
    df['quantity'] = df['quantity'].astype(int)
    df['price'] = df['price'].astype(float)
    
    return df

In [14]:
def calculate_revenue(df):
    quantities = df['quantity'].to_numpy()
    prices = df['price'].to_numpy()
    revenue = np.multiply(quantities, prices)
    
    # Add revenue column to DataFrame
    df['revenue'] = revenue
    return df

In [15]:
def transform_dates(df):
    df['order_date'] = pd.to_datetime(df['order_date'])
    # Create new columns for year, month, and day
    df['year'] = df['order_date'].dt.year
    df['month'] = df['order_date'].dt.month
    df['day'] = df['order_date'].dt.day
    return df

In [16]:
def plot_monthly_sales(df):
    # Group by month and calculate total revenue for each month
    monthly_sales = df.groupby('month')['revenue'].sum()
    
    # Plotting with Pandas
    monthly_sales.plot(kind='bar', figsize=(10, 6), color='skyblue')
    plt.title('Monthly Sales Trend')
    plt.xlabel('Month')
    plt.ylabel('Total Revenue ($)')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.savefig('monthly_sales_trend.png')
    # plt.show()
    plt.close()

In [17]:
def create_database_schema():
    # Connect to MySQL
    conn = mysql.connector.connect(
        host="localhost",
        user="angela",
        password="angela@user"
    )
    cursor = conn.cursor()
    
    cursor.execute("CREATE DATABASE IF NOT EXISTS shopease")
    cursor.execute("USE shopease")
   
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS sales (
            order_id INT PRIMARY KEY,
            customer_id INT,
            order_date DATE,
            product_id INT,
            quantity INT,
            revenue FLOAT,
            year INT,
            month INT,
            day INT
        )
    """)
    
    conn.commit()
    conn.close()

In [18]:
def load_to_database(df):
    # Connect to the MySQL database
    conn = mysql.connector.connect(
        host="localhost",
        user="angela",
        password="angela@user",
        database="shopease"
    )
    cursor = conn.cursor()
    
    # Insert data row by row into `sales` table
    insert_query = """
        INSERT INTO sales (order_id, customer_id, order_date, product_id, quantity, revenue, year, month, day)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Track any failed insertions
    failed_rows = []
    
    for _, row in df.iterrows():
        try:
            cursor.execute(insert_query, (
                row['order_id'], row['customer_id'], row['order_date'],
                row['product_id'], row['quantity'], row['revenue'],
                row['year'], row['month'], row['day']
            ))
        except Exception as e:
            print(f"Failed to insert row {row['order_id']}: {e}")
            failed_rows.append(row)
    
    conn.commit()
    conn.close()
    

    print(f"Inserted {len(df) - len(failed_rows)} rows successfully.")

In [19]:
def main():

    print("1. Loading data...")
    df = load_data()
    
    print("2. Cleaning data...")
    df = clean_data(df)
    
    print("3. Calculating revenue...")
    df = calculate_revenue(df)
    
    print("4. Transforming dates...")
    df = transform_dates(df)
    
    print("5. Creating visualization...")
    plot_monthly_sales(df)
    
    print("6. Creating database schema...")
    create_database_schema()
    
    print("7. Loading data to database...")
    load_to_database(df)
    
    print("\nSample of processed data:")
    print(df.head())
    
    return df

In [20]:
if __name__ == "__main__":
    processed_df = main()

1. Loading data...
2. Cleaning data...
3. Calculating revenue...
4. Transforming dates...
5. Creating visualization...
6. Creating database schema...
7. Loading data to database...
Inserted 570 rows successfully.

Sample of processed data:
   order_id  customer_id order_date  product_id  quantity product_name  \
0         1           20 2024-11-01           3         2       Tablet   
1         2           36 2024-11-01           1         1       Laptop   
2         3            1 2024-11-01           8         3   Headphones   
3         4           36 2024-11-02           6         1   Smartwatch   
4         5           14 2024-11-02          12         2         Desk   

      category   price  revenue  year  month  day  
0  Electronics   450.0    900.0  2024     11    1  
1  Electronics  1200.0   1200.0  2024     11    1  
2  Electronics   150.0    450.0  2024     11    1  
3  Electronics   200.0    200.0  2024     11    2  
4    Furniture   300.0    600.0  2024     11    2  
